In [1]:
## Import necessary libraries 

import os 
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import keras
import datetime as dt
import matplotlib.pyplot as plt 
import urllib.request, json 
from sklearn.preprocessing import MinMaxScaler
from pandas_datareader import data

Using TensorFlow backend.
